In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objects as go #for visualisation

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv")

In [ ]:
data.head()

In [ ]:
data.info()

### removing irrelevant columns for the current scenario

In [ ]:
data.drop(["source_name", "source_website", "iso_code", "daily_vaccinations_raw"], axis = 1, inplace = True)

## Prepading data for visualisation

In [ ]:
data.date = pd.to_datetime(data.date)

In [ ]:
data.sort_values(['country', 'date'], 
              ascending = [True, True], inplace = True)

In [ ]:
data.tail()

### Droping countries with less than 30 days of data

In [ ]:
countries_to_drop = []
for c in data.country.unique():
    if len(data[data['country'] == c]) < 30:
       countries_to_drop.append(c) 

In [ ]:
for c in countries_to_drop:
    data = data.drop(data[data.country == c].index)

We can now forward fill the below columns for each countires

In [ ]:
cols_last_val_imputation = ["total_vaccinations", "people_vaccinated", "people_fully_vaccinated", "total_vaccinations_per_hundred", 
                            "people_vaccinated_per_hundred", "people_fully_vaccinated_per_hundred", "vaccines"]

In [ ]:
data.shape

In [ ]:
for c in data.country.unique():
    data.loc[data['country'] == c, cols_last_val_imputation] = data.loc[data['country'] == c, cols_last_val_imputation].fillna(method='ffill')

In [ ]:
data.head()

Filling in the na values with 0 for the non forward filled columns

In [ ]:
data.fillna(0.0, inplace = True)

In [ ]:
data.head()

In [ ]:
num_cols = ["total_vaccinations", "people_vaccinated", "people_fully_vaccinated", "daily_vaccinations",
            "total_vaccinations_per_hundred", "people_vaccinated_per_hundred", 
            "people_fully_vaccinated_per_hundred", "daily_vaccinations_per_million"]

In [ ]:
global_data = data.groupby(['date'])[num_cols].mean().reset_index()

In [ ]:
global_data.head()

## Comparing each infomation of countires with the Global mean

In [ ]:
def plot_feature(feature):
    
    countries = sorted(set(data["country"]))

    fig=go.Figure()

    country_plot_names = []
    buttons=[]

    default_country = "Ireland"

    for country in countries:
        data_country = data[(data["country"]==country)]
        data_global = global_data

        # We have two traces we're plotting per state: a boxplot of the submission quartiles, and a line with the current data to-date
        fig.add_trace(go.Scatter(name = country, x=data_country["date"], y=data_country[feature],line={}, visible=(country==default_country)))
        fig.add_trace(go.Scatter(name = 'Global',x=data_global["date"], y=data_global[feature], line={}))

        country_plot_names.extend([country]*2)

    for country in countries:
        buttons.append(dict(method='update',
                            label=country,
                            args = [{'visible': [country==r for r in country_plot_names]}]))

    # Add dropdown menus to the figure
    fig.update_layout(showlegend=True, updatemenus=[{"buttons": buttons, "direction": "down", "active": countries.index(default_country), "showactive": True, "x": 0.5, "y": 1.15}])
    fig.show()

In [ ]:
plot_feature('total_vaccinations')

In [ ]:
plot_feature('total_vaccinations_per_hundred')